### Figure 1: Predicted images

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import colors
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import metpy.plots.ctables
from utils.data import load_tc_data
from utils.plot import make_cmap

# TODO: because i'm using imshow the lats need to be flipped - normally would be fine but because lats are flipped and imshow doesn't know this
# the TCs are plotted spinning the wrong way

sns.set_style("white")
sns.set_palette(sns.color_palette("Paired"))
sns.set_palette(sns.color_palette("Set2"))

In [2]:
# define cmap
precip_cmap,precip_norm = make_cmap()

In [3]:
def regrid(array):
        hr_array = np.zeros((100,100))
        for i in range(10):
                for j in range(10):
                        i1 = i*10
                        i2 = (i+1)*10
                        j1 = j*10
                        j2 = (j+1)*10
                        hr_array[i1:i2,j1:j2] = array[i,j]
        return hr_array

In [4]:
def plot_predictions(inputs,inputs_x,pred_cnn,pred_cnn_x,pred_vaegan,pred_vaegan_x,pred_gan,pred_gan_x,real,real_x,storms,range_,plot='save'):
	if plot == 'save':
			fig, axes = plt.subplots(n, m, figsize=(5*m, 5*n), sharey=True)
	else:
			print('show')
			fig, axes = plt.subplots(n, m, figsize=(2*m, 2*n), sharey=True)

	pal = sns.cubehelix_palette(10, rot=-.25, light=.7,as_cmap=True)

	for i in range(m):
		j = 0
		
		storm = storms[i]
		
		print('storm max: ',real[storm].max())
		print('cnn max: ',pred_cnn[storm].max())
		print('vaegan max: ',pred_vaegan[storm].max())
		print('gan max: ',pred_gan[storm].max())
		print('inputs max: ', np.nanmax(inputs[storm]))
		# norm = colors.Normalize(*range_)
		norm = precip_norm
		if i in [0,1,2]:
			axes[j,i].imshow(regrid(inputs[storm]), interpolation='nearest', norm=norm,extent=None,cmap=precip_cmap)
			axes[j+1,i].imshow(pred_cnn[storm], interpolation='nearest',norm=norm, extent=None,cmap=precip_cmap)
			axes[j+2,i].imshow(pred_vaegan[storm], interpolation='nearest',norm=norm, extent=None,cmap=precip_cmap)
			axes[j+3,i].imshow(pred_gan[storm], interpolation='nearest',norm=norm, extent=None,cmap=precip_cmap)
			im = axes[j+4,i].imshow(real[storm], interpolation='nearest',norm=norm, extent=None,cmap=precip_cmap)
		else:
			axes[j,i].imshow(regrid(inputs_x[storm]), interpolation='nearest', norm=norm,extent=None,cmap=precip_cmap)
			axes[j+1,i].imshow(pred_cnn_x[storm], interpolation='nearest',norm=norm, extent=None,cmap=precip_cmap)
			axes[j+2,i].imshow(pred_vaegan_x[storm], interpolation='nearest',norm=norm, extent=None,cmap=precip_cmap)
			axes[j+3,i].imshow(pred_gan_x[storm], interpolation='nearest',norm=norm, extent=None,cmap=precip_cmap)
			im = axes[j+4,i].imshow(real_x[storm], interpolation='nearest',norm=norm, extent=None,cmap=precip_cmap)

		axes[j,i].set(xticklabels=[])
		axes[j,i].set(yticklabels=[])
		axes[j+1,i].set(xticklabels=[])
		axes[j+1,i].set(yticklabels=[])
		axes[j+2,i].set(xticklabels=[])
		axes[j+2,i].set(yticklabels=[])
		axes[j+3,i].set(xticklabels=[])
		axes[j+3,i].set(yticklabels=[])
		axes[j+4,i].set(xticklabels=[])
		axes[j+4,i].set(yticklabels=[])


		axes[j,0].set_ylabel('LR input',rotation=0,horizontalalignment='left',fontsize=48,labelpad=240,fontweight='bold',color='#56a1d7')
		axes[j+1,0].set_ylabel('U-NET',rotation=0,horizontalalignment='left',fontsize=48,labelpad=240,fontweight='bold',color='#56a1d7')
		axes[j+2,0].set_ylabel('VAEGAN',rotation=0,horizontalalignment='left',fontsize=48,labelpad=240,fontweight='bold',color='#56a1d7')
		axes[j+3,0].set_ylabel('GAN',rotation=0,horizontalalignment='left',fontsize=48,labelpad=240,fontweight='bold',color='#56a1d7')
		axes[j+4,0].set_ylabel('HR obs',horizontalalignment='left',rotation=0,fontsize=48,labelpad=240,fontweight='bold',color='#56a1d7')

		axes[4,j].set_xlabel('Maria (2017; NA)',fontsize=24,labelpad=5)
		axes[4,j+1].set_xlabel('Haiyan (2013; WP)',fontsize=24,labelpad=5)
		axes[4,j+2].set_xlabel('Barbara (1995; EP)',fontsize=24,labelpad=5)
		axes[4,j+3].set_xlabel('Georges (1998; NA)',fontsize=24,labelpad=5)
		axes[4,j+4].set_xlabel('Daman (1992; SP)',fontsize=24,labelpad=5)
		axes[4,j+5].set_xlabel('Haleh (2019; SI)',fontsize=24,labelpad=5)

	fig.subplots_adjust(right=0.8)
	# cbar_ax = fig.add_axes([1, 0.15, 0.05, 0.7])
	cbar_ax = fig.add_axes([1, 0.031, 0.03, 0.936])
	cb = fig.colorbar(im, cax=cbar_ax,shrink=0.01)
	cb.ax.tick_params(labelsize=36)
	fig.tight_layout()

	if plot == 'save':
			plt.savefig('figure_1.png',bbox_inches='tight')
			plt.clf()
	else:
			plt.show()
	

In [10]:
# load data
real,inputs,pred_cnn,pred_vaegan,pred_gan,pred_vaegan_ensemble,pred_gan_ensemble,meta = load_tc_data(set='validation',results='test')
real_x,inputs_x,pred_cnn_x,pred_vaegan_x,pred_gan_x,pred_vaegan_ensemble_x,pred_gan_ensemble_x,meta_extreme_valid = load_tc_data(set='extreme_valid',results='test')

meta = pd.read_csv('/user/work/al18709/tc_data_mswep/valid_meta.csv')
meta_extreme = pd.read_csv('/user/work/al18709/tc_data_mswep/extreme_valid_meta.csv')
meta_extreme_test = pd.read_csv('/user/work/al18709/tc_data_mswep/extreme_test_meta.csv')
meta_test = pd.read_csv('/user/work/al18709/tc_data_mswep/test_meta.csv')
meta_train = pd.read_csv('/user/work/al18709/tc_data_mswep/train_meta.csv')
meta_valid = pd.read_csv('/user/work/al18709/tc_data_mswep/valid_meta.csv')

In [6]:

plot = 'save'
n = 5
m = 6

# make colours
range_ = (0, 20)
storms = [4077, 4078, 4079, 4080, 4081, 4082]

# maria 8453 (8452 for unet) 8463
# haiyan 9820, 9828,  9845, 9840 9815, 9816, 9822 9809
# barb 1347 1375 1399
# georges 4082

storms = [8453,9845,1375,4128,5358,747]
# storms_extremes = [4082,5349,763]
plot_predictions(inputs,inputs_x,pred_cnn,pred_cnn_x,pred_vaegan,pred_vaegan_x,pred_gan,pred_gan_x,real,real_x,storms,range_)


storm max:  45.25
cnn max:  29.72098
vaegan max:  43.44794464111328
gan max:  51.5527229309082
inputs max:  20.504125595092773
storm max:  58.875
cnn max:  15.141447
vaegan max:  65.58946228027344
gan max:  61.957401275634766
inputs max:  31.177553176879883
storm max:  48.3125
cnn max:  37.28275
vaegan max:  51.87416458129883
gan max:  60.7703971862793
inputs max:  29.4201602935791
storm max:  44.375
cnn max:  31.241987
vaegan max:  39.170387268066406
gan max:  42.21123123168945
inputs max:  25.278505325317383
storm max:  59.9375
cnn max:  41.355465
vaegan max:  59.43865966796875
gan max:  55.63440704345703
inputs max:  31.395225524902344
storm max:  44.8125
cnn max:  38.602913
vaegan max:  51.64292907714844
gan max:  47.49197769165039
inputs max:  20.156835556030273


/user/work/al18709/.conda/envs/aquatic/lib/python3.7/site-packages/ipykernel_launcher.py:65: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.


<Figure size 2160x1800 with 0 Axes>

In [14]:
# print(meta.shape)
# print(real.shape) 

# print(np.sum(meta['sid'] == '1992044S09181'))
# print(np.sum(meta_extreme['sid'] == '1992044S09181'))
# print(np.sum(meta_test['sid'] == '1992044S09181'))
# print(np.sum(meta_extreme_test['sid'] == '1992044S09181'))
# print(np.sum(meta_train['sid'] == '1992044S09181'))


# print(meta_extreme['sid'].values)
# for i,storm in enumerate(meta_extreme['sid'].values):
# 	pre_storm = meta_extreme['sid'].values[i-1]
# 	if storm != pre_storm:
# 		print(storm)

# Dorian in the test set
# Isaac (2012) '2012234N16315' training set NA
# Amphan 2020136N10088 ****** lost NI
# Harvey 2017228N14314 train NA
# Irma 2017242N16333 train NA
# Idai 2019063S18038 train SIO 
# Winston 2016041S14170 test SP
# Matthew 2016273N13300 extreme_test NA
# Jalawat 2012263N15141 extreme_test WP
# irene 2011233N15301 valid
# isaac 2012234N16315 train


# EP NA WP
# NIO SIO SP

# Maria 2017260N12310 validation NA
# Haiyan 2013306N07162 validation WP
# Barbara 1995189N11257 validation EP
print(np.where(meta_extreme['sid'] == '2012234N16315'))
print(np.where(meta_valid['sid'] == '2012234N16315'))
print(np.where(meta_train['sid'] == '2012234N16315'))
print(np.where(meta_test['sid'] == '2012234N16315'))
print(np.where(meta_extreme_test['sid'] == '2012234N16315'))
print(np.where(meta_extreme_valid['sid'] == '2012234N16315'))

# Georges 1998259N10335 extreme valid NA
# Daman 1992044S09181 extreme valid SP
# Haleh 2019060S10078 extreme valid SIO

(array([], dtype=int64),)
(array([], dtype=int64),)
(array([36367, 36368, 36369, 36370, 36371, 36372, 36373, 36374, 36375,
       36376, 36377, 36378, 36379, 36380, 36381, 36382, 36383, 36384,
       36385, 36386, 36387, 36388, 36389, 36390, 36391, 36392, 36393,
       36394, 36395, 36396, 36397, 36398, 36399, 36400, 36401, 36402,
       36403, 36404, 36405, 36406, 36407, 36408, 36409, 36410, 36411,
       36412, 36413, 36414, 36415, 36416, 36417, 36418, 36419, 36420,
       36421, 36422, 36423, 36424, 36425, 36426]),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)


In [8]:
print(pred_cnn)

[[[ 0.5377605   0.42959526  0.32142994 ...  0.39529592  0.4091882
    0.42308044]
  [ 0.5454536   0.5268012   0.5081488  ...  0.30019972  0.3330531
    0.3659064 ]
  [ 0.5531465   0.6240071   0.6948676  ...  0.20510352  0.25691792
    0.30873233]
  ...
  [ 0.42230648  0.3376853   0.2530641  ...  0.08054145  0.22475798
    0.36897457]
  [ 0.34279606  0.3038701   0.2649442  ...  0.18767123  0.31103966
    0.43440807]
  [ 0.26328564  0.270055    0.27682427 ...  0.2948011   0.39732137
    0.49984166]]

 [[ 0.1530547   0.10673091  0.06040713 ...  0.2915675   0.29270607
    0.29384452]
  [ 0.19114704  0.19671716  0.2022873  ...  0.2493409   0.27411994
    0.29889894]
  [ 0.22923937  0.28670344  0.34416747 ...  0.20711435  0.25553384
    0.3039533 ]
  ...
  [ 0.15043722  0.15032277  0.15020831 ...  0.14383714  0.20800506
    0.27217302]
  [ 0.15043722  0.14471464  0.13899207 ...  0.18423365  0.25864002
    0.33304644]
  [ 0.15043722  0.13910653  0.12777583 ...  0.2246301   0.30927503
    0.39

In [11]:
print(pred_cnn_x.shape)
print(pred_vaegan_x.shape)


(5425, 100, 100)
(5425, 100, 100)


In [18]:
print(real.shape)
print(real_x.shape)

(16253, 100, 100)
(5425, 100, 100)
